In [1]:
import torch
import numpy as np

In [2]:
from models import Generator

In [5]:
noise_dimension=5
output_dimension=20

In [6]:
G = Generator(noise_dimension=noise_dimension, output_dimension=output_dimension)

In [7]:
z = torch.randn(noise_dimension)